In [7]:
len(ppg_inp), len(lowess(ppg_inp)), np.sum(np.isnan(ppg_inp))

(2000, 1921, 79)

In [8]:
import os, sys
import pickle
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 0.2가 제일 잘 없앴음
def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T


LEN_INPUT = 20
OVERLAP = 10
SRATE = 100
n_aug = int((60-LEN_INPUT)/OVERLAP) + 1  


df_preprocess = pickle.load(open('cache/preprocess/df_preprocess', 'rb'))
for f_num, rows in df_preprocess.iterrows():
    caseid = rows['caseid']
    print(f'\n###Input{f_num}/{len(df_preprocess)}: {caseid}###')
    

    # vital data 불러오기    
    file_path = 'input_60s'
    vals = np.load(f'{file_path}/{caseid}.npz')


    # 20초 단위로 끊기
    for i in range(n_aug):
        print('  n_segment', i+1, end='')
        start_idx = i*OVERLAP*SRATE # 500i
        end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000
        
        # non-event data
        if rows[str(i+1)][0]:
            print(' passed...lowess filtering...', end='')
            
            save_path = f'cache/lowess_filtered/input20s-10s/{caseid}_n{i+1}.npz'
            if os.path.exists(save_path):
                print('already exists', end='')
                continue

            ppg_inp = vals['nPPG'][start_idx:end_idx]
            ecg_inp = vals['nECG'][start_idx:end_idx]
            
            ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
            ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
            
            # lowess filter 적용
            ppg_input = ppg_inp - lowess(ppg_inp)
            ecg_input = ecg_inp - lowess(ecg_inp)
            
            np.savez(save_path, ECG = ecg_input, PPG = ppg_input)
            print('done', end=' ')

    print('')
    for i in range(n_aug):
        print('  segment', i+1, end='')
        start_idx = i*OVERLAP*SRATE # 500i
        end_idx = (i*OVERLAP + LEN_INPUT)*SRATE # 500i + 1000
            
        # event data
        if rows[str(i+n_aug+1)][0]:
            print(' passed...lowess filtering...', end='')
            
            save_path = f'cache/lowess_filtered/input20s-10s/{caseid}_{i+1}.npz'
            if os.path.exists(save_path):
                print('already exists', end='')
                continue

            ppg_inp = vals['PPG'][start_idx:end_idx]
            ecg_inp = vals['ECG'][start_idx:end_idx]
            
            ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
            ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
            
            # lowess filter 적용
            ppg_input = ppg_inp - lowess(ppg_inp)
            ecg_input = ecg_inp - lowess(ecg_inp)
            
            np.savez(save_path, ECG = ecg_input, PPG = ppg_input) 
            print('done', end=' ')


###Input0/2684: 5974###
  n_segment 1 passed...lowess filtering...already exists  n_segment 2 passed...lowess filtering...already exists  n_segment 3 passed...lowess filtering...already exists  n_segment 4 passed...lowess filtering...already exists  n_segment 5 passed...lowess filtering...already exists
  segment 1 passed...lowess filtering...already exists  segment 2 passed...lowess filtering...already exists  segment 3 passed...lowess filtering...already exists  segment 4 passed...lowess filtering...already exists  segment 5 passed...lowess filtering...already exists
###Input1/2684: 801###
  n_segment 1 passed...lowess filtering...already exists  n_segment 2 passed...lowess filtering...already exists  n_segment 3 passed...lowess filtering...already exists  n_segment 4 passed...lowess filtering...already exists  n_segment 5 passed...lowess filtering...already exists
  segment 1  segment 2  segment 3  segment 4  segment 5
###Input2/2684: 2790###
  n_segment 1 passed...lowess filtering

In [12]:
!pip install psutil

In [13]:
import psutil

psutil.cpu_percent()

ModuleNotFoundError: No module named 'psutil'

In [18]:
#%load_ext autoreload
#%autoreload 2

import learn

learn.load_vital_data('vital_to_np')

loading vital data...
1/2699(3)...already existing
2/2699(8)...already existing
3/2699(9)...already existing
4/2699(13)...already existing
5/2699(14)...already existing
6/2699(16)...already existing
7/2699(17)...already existing
8/2699(19)...already existing
9/2699(20)...already existing
10/2699(22)...already existing
11/2699(28)...already existing
12/2699(29)...already existing
13/2699(30)...already existing
14/2699(35)...already existing
15/2699(36)...already existing
16/2699(40)...already existing
17/2699(42)...already existing
18/2699(44)...already existing
19/2699(45)...already existing
20/2699(46)...already existing
21/2699(47)...already existing
22/2699(48)...already existing
23/2699(50)...already existing
24/2699(54)...already existing
25/2699(57)...already existing
26/2699(59)...already existing
27/2699(67)...already existing
28/2699(69)...already existing
29/2699(70)...already existing
30/2699(72)...already existing
31/2699(74)...already existing
32/2699(75)...already existin

IndexError: index 2910 is out of bounds for axis 0 with size 2910